# Deep Learning

The following sample codes are examples of CNN, LSTM and a combined network with DNN on static features and LSTM on time series features.

## 1. CNN

In [ ]:
# Convert train data to np array, reshape to 4 dimensional.
from sklearn import preprocessing
ActiveTran2 = preprocessing.normalize(ActiveTran1) #range 0-1
ActiveTran2 = pd.DataFrame(ActiveTran2, columns = ActiveTran1.columns)
ActiveTran3 = ActiveTran2.values

tran = []
for i in range(len(TrainLab)):
    age = ActiveTran2.loc[i:i+5,['Age']]
    tm = ActiveTran2.loc[i:i+5,['TenureMonth']]
    s = ActiveTran2.loc[i:i + 5, ['NoOfSavingsP']]
    l = ActiveTran2.loc[i:i + 5, ['NoOfLoans']]
    dd = ActiveTran2.loc[i:i + 5, ['NumberofDirectDeposits']]
    sd = ActiveTran2.loc[i:i + 5, ['SumofDirectDeposits']]
    bp = ActiveTran2.loc[i:i + 5, ['NumberofBillPayTransactions']]
    dc = ActiveTran2.loc[i:i + 5, ['NumberofDebitCardTransactions']]
    ib = ActiveTran2.loc[i:i + 5, ['NumberofTransactionsConductedinBranch']]
    fc = ActiveTran2.loc[i:i + 5, ['FEESCHARGED']]
    sf = ActiveTran2.loc[i:i + 5, ['SUMofFEESCHARGED']]
    tt = ActiveTran2.loc[i:i + 5, ['NumberofTotalTransactions']]
    ol = ActiveTran2.loc[i:i + 5, ['OnlineTran']]
    py = ActiveTran2.loc[i:i + 5, ['NumberofPayments']]
    sp = ActiveTran2.loc[i:i + 5, ['SumofPaymentAmount']]
    at = ActiveTran2.loc[i:i + 5, ['AllTran']]
    a = np.vstack((age, tm, s, l, dd, sd, bp, dc, ib, fc, sf, tt, ol, py, sp, at))
    a = a.tolist()
    tran.append(a)
TrainTran = np.array(tran)
TrainTran = np.reshape(tran,(TrainTran.shape[0],16,6,1))

#Convert Target to categorical
from keras.utils.np_utils import to_categorical
Y = TrainLab[['Loyalty']].values
Y = to_categorical(Y,5)

In [ ]:
# Building CNN

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.utils import np_utils
from keras import backend as K
#K.set_image_data_format('channels_last')
from keras.optimizers import Nadam

#Define model architecture
model = Sequential()

model.add(Convolution2D(64, 3, 3, activation='relu', input_shape=(16, 6, 1)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(MaxPooling2D(pool_size=(1, 1)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Dropout(0.25))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Dropout(0.25))
model.add(MaxPooling2D((1,1), strides=(1,1)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

# Compile model, use cost sensitive learning
model.compile(loss=WeightedCategoricalCrossentropy(cost_matrix), optimizer='adam', metrics=[tf.keras.metrics.CategoricalCrossentropy()])


#Fit model on training data
model.fit(TrainTran1, Train_Y3a, batch_size=32, epochs = 100, verbose=2)

#Evaluate model on test data
score = model.evaluate(TestTran1, Test_Y, verbose=2) #accuracy = 0.4685 (1 dense layer)
#0.4302 (without dropout 0.5), 0.4691 (with dropout 0.5), Add more conv2d layers 0.4681, add dropout layer 0.4731
#Change batch size no difference

history = model.fit(TrainTran1, Y, validation_data=(TestTran1, Test_Y), batch_size=32, epochs=50, verbose=2)

# Get training and test loss histories
training_loss = history.history['loss']
test_loss = history.history['val_loss']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show();

## 2. LSTM

In [ ]:
# Data Preparation, convert train data to 3 dimensional np array
from sklearn import preprocessing
ActiveTran2 = preprocessing.normalize(ActiveTran1) #range 0-1
ActiveTran2 = pd.DataFrame(ActiveTran2, columns = ActiveTran1.columns)

tran = []
for i in range(len(TrainLab)):
    age = ActiveTran2.loc[i:i+5,['Age']]
    tm = ActiveTran2.loc[i:i+5,['TenureMonth']]
    s = ActiveTran2.loc[i:i + 5, ['NoOfSavingsP']]
    l = ActiveTran2.loc[i:i + 5, ['NoOfLoans']]
    dd = ActiveTran2.loc[i:i + 5, ['NumberofDirectDeposits']]
    sd = ActiveTran2.loc[i:i + 5, ['SumofDirectDeposits']]
    bp = ActiveTran2.loc[i:i + 5, ['NumberofBillPayTransactions']]
    dc = ActiveTran2.loc[i:i + 5, ['NumberofDebitCardTransactions']]
    ib = ActiveTran2.loc[i:i + 5, ['NumberofTransactionsConductedinBranch']]
    fc = ActiveTran2.loc[i:i + 5, ['FEESCHARGED']]
    sf = ActiveTran2.loc[i:i + 5, ['SUMofFEESCHARGED']]
    tt = ActiveTran2.loc[i:i + 5, ['NumberofTotalTransactions']]
    ol = ActiveTran2.loc[i:i + 5, ['OnlineTran']]
    py = ActiveTran2.loc[i:i + 5, ['NumberofPayments']]
    sp = ActiveTran2.loc[i:i + 5, ['SumofPaymentAmount']]
    at = ActiveTran2.loc[i:i + 5, ['AllTran']]
    a = np.hstack((age, tm, s, l, dd, sd, bp, dc, ib, fc, sf, tt, ol, py, sp, at))
    a = a.tolist()
    tran.append(a)
TrainTran = np.array(tran)
TrainTranL = np.reshape(tran,(TrainTran.shape[0],6,16))

#Convert Target to categorical
from keras.utils.np_utils import to_categorical
Test_Y3 = TestLab[['Loyalty1']].values
Test_Y3 = to_categorical(Test_Y3,3)

Train_Y3 = TrainLab[['Loyalty1']].values
Train_Y3 = to_categorical(Train_Y3,3)
Y_train = Train_Y3.reshape(-1,1)

In [ ]:
# Building LSTM

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
import tensorflow as tf
import keras as K

n_timesteps, n_features, n_outputs = TrainTranL.shape[1], TrainTranL.shape[2], Train_Y3.shape[1]

verbose, epochs, batch_size = 2, 200, 32

model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_timesteps,n_features)))
#model.add(LSTM(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalCrossentropy()])

# fit network
history = model.fit(TrainTranL, Train_Y3, epochs=epochs, batch_size=batch_size, verbose=verbose)
# evaluate model
accuracy = model.evaluate(TestTranL, Test_Y3, batch_size=batch_size, verbose=2)

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

#Result of 150 epochs similar result with 200 epoch
rounded_predictions = model.predict_classes(TestTranL, batch_size=batch_size, verbose=verbose)
rounded_labels=np.argmax(Test_Y3, axis=1)
print(classification_report(rounded_labels, rounded_predictions))

# Get training and test loss histories
training_loss = history.history['loss']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show();


## 3. Combined Network: DNN (static features) and LSTM (time series features)

In [ ]:
#Data Preprocessing
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
ActiveTran2 = scaler.fit_transform(ActiveTran1b)
ActiveTran2 = pd.DataFrame(ActiveTran2, columns = ActiveTran1b.columns)

tran = []
for i in range(len(TrainSet3a)):
    dd = ActiveTran2.loc[i:i + 5, ['NumberofDirectDeposits']]
    sd = ActiveTran2.loc[i:i + 5, ['SumofDirectDeposits']]
    bp = ActiveTran2.loc[i:i + 5, ['NumberofBillPayTransactions']]
    dc = ActiveTran2.loc[i:i + 5, ['NumberofDebitCardTransactions']]
    ib = ActiveTran2.loc[i:i + 5, ['NumberofTransactionsConductedinBranch']]
    fc = ActiveTran2.loc[i:i + 5, ['FEESCHARGED']]
    sf = ActiveTran2.loc[i:i + 5, ['SUMofFEESCHARGED']]
    tt = ActiveTran2.loc[i:i + 5, ['NumberofTotalTransactions']]
    ol = ActiveTran2.loc[i:i + 5, ['OnlineTran']]
    py = ActiveTran2.loc[i:i + 5, ['NumberofPayments']]
    sp = ActiveTran2.loc[i:i + 5, ['SumofPaymentAmount']]
    at = ActiveTran2.loc[i:i + 5, ['AllTran']]
    a = np.hstack((dd, sd, bp, dc, ib, fc, sf, tt, ol, py, sp, at))
    a = a.tolist()
    tran.append(a)
TrainTS = np.array(tran)

#Convert Target to categorical
from keras.utils.np_utils import to_categorical
Test_Y3 = TestLab[['Loyalty1']].values
Test_Y3 = to_categorical(Test_Y3,3)


In [ ]:
#LSTM for time series inputs
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
import tensorflow as tf
from keras.layers import Input
from keras.layers.merge import concatenate
from keras.models import Model

n_timesteps, n_features = TrainTS.shape[1], TrainTS.shape[2]
X1 = TrainTS

#First input model
visible1 = Input(shape=(n_timesteps, n_features))
LSTM1 = LSTM(48, activation='relu')(visible1)
LSTM1 = Dropout(0.5)(LSTM1)
LSTM1 = Flatten()(LSTM1)

In [ ]:
## Prepare static predictors
Train = Train_XML.values
Test = Test_XML.values

#Second input model
X2 = Train

visible2 = Input(shape=Train.shape[1])
DNN1 = Dense(72, activation='relu')(visible2)
DNN1 = Dropout(0.5)(DNN1)
DNN1 = Dense(24, activation='relu')(DNN1)
DNN1 = Flatten()(DNN1)


In [ ]:
# merge input models
n_outputs = Train_Y3.shape[1]

Merge1 = concatenate([LSTM1, DNN1])
DNN2 = Dense(50, activation='relu')(Merge1)
DNN2 = Dropout(0.5)(DNN2)
DNN2 = Dense(25, activation='relu')(DNN2)
DNN2 = Dropout(0.5)(DNN2)
output = Dense(n_outputs, activation='softmax')(DNN2)
model = Model(inputs=[visible1, visible2], outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalCrossentropy()])

history = model.fit([X1, X2], Train_Y3, epochs=125, verbose=2)

from sklearn.metrics import classification_report

#Result of 100 epochs
verbose, batch_size = 2, 32
rounded_predictions = model.predict([TestTS, Test], batch_size=batch_size, verbose=verbose)
prediction = np.argmax(rounded_predictions, axis=1)
rounded_labels=np.argmax(Test_Y3, axis=1)
print(classification_report(rounded_labels, prediction, digits=4))

# Visualize loss history
training_loss = history.history['loss']
epoch_count = range(1, len(training_loss) + 1)
plt.plot(epoch_count, training_loss, 'r--')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show();
